In [1]:
import yfinance as yf
import pandas as pd
from plotly import express as px
import plotly.io as pio
pio.templates.default = "plotly_dark"

# EPAM vs Peers vs Market Analysis

In [2]:
tickers = [
    'EPAM', 'DOX', 'GLOB', 'DXC', 'GIB', 'IBM', 'GOOG', 'AMZN', 'NVDA', 'TSLA',
    'SPY'
]

# Extract adjusted closed prices
price_df = yf.download(
    tickers = tickers,
    start = '2021-01-01'
)['Adj Close']
start_date = price_df['EPAM'].index.min()

# Extract returns
annual_returns_df = price_df.pct_change(255).dropna()

# Find correlations with EPAM's stock
cols = [col for col in price_df.columns if col != 'EPAM']
corr_df = price_df[cols].rolling(21).corr(price_df['EPAM']).dropna()

# Generate index
index_df = price_df/price_df.loc[start_date, :] * 100

[*********************100%***********************]  11 of 11 completed


In [3]:
# Plot prices
px.line(
    data_frame = price_df,
    title = 'Historical Daily Adj. Close Prices',
    height = 600,
    labels = {'variable': 'Ticker', 'value': 'USD'}
)

In [4]:
# Plot index
px.line(
    data_frame = index_df,
    title = 'Historical Index Based on Daily Adj. Close Prices',
    height = 600,
    labels = {'variable': 'Ticker', 'value': 'Unit'}
)

In [5]:
price_df.pct_change(1).dropna().mean()

AMZN    0.000022
DOX     0.000497
DXC     0.000099
EPAM    0.000276
GIB     0.000488
GLOB    0.000362
GOOG    0.000828
IBM     0.000605
NVDA    0.002486
SPY     0.000387
TSLA    0.000689
dtype: float64

In [6]:
# Plot annual returns
fig = px.line(
    data_frame = annual_returns_df,
    title = 'Historical 255-day returns',
    height = 600,
    labels = {'variable': 'Ticker', 'value': 'Pct'}
)
fig.update_layout(
    yaxis_tickformat = '1%'
)

In [7]:
# Plot monthly moving correlation
fig = px.line(
    data_frame = corr_df,
    title = 'Historical Monthly Moving Correlation with EPAM Price',
    height = 600,
    labels = {'variable': 'Ticker', 'value': 'Pct'}
)
fig.update_layout(
    yaxis_tickformat = '1%'
)

# S&P500 Analysis
***

In [37]:
# Extract adjusted closed prices
tickers = ['COP=X', 'SPY']
price_df = yf.download(
    tickers = tickers,
    start = '1900-01-01'
)['Adj Close']
price_df.dropna(subset=['SPY'], inplace=True)

mask = price_df['COP=X'] < 1000
price_df.loc[mask, 'COP=X'] =  None
price_df.fillna(method='ffill', inplace=True)

price_df['SPY-COP'] = price_df['SPY'] * price_df['COP=X']

# Extract returns
annual_returns_df = price_df.pct_change(255).dropna()

[*********************100%***********************]  2 of 2 completed


In [38]:
# Plot annual returns
fig = px.line(
    data_frame = annual_returns_df,
    title = 'Historical 255-day returns',
    height = 600,
    labels = {'variable': 'Ticker', 'value': 'Pct'}
)
fig.update_layout(
    yaxis_tickformat = '1%'
)

In [39]:
annual_returns_df.describe()

,COP=X,SPY,SPY-COP
count,4943.000000,4943.000000,4943.000000
mean,0.033018,0.111071,0.138014
std,0.144149,0.160682,0.176915
min,-0.270978,-0.480853,-0.387135
25%,-0.069505,0.046740,0.029128
50%,0.005072,0.129723,0.127615
75%,0.104687,0.192611,0.241579
max,0.722543,0.804576,0.821426


In [40]:
# Generate S&P 500 index
start_date = '2020-01-02'
index_df = price_df.loc[start_date:, :]/price_df.loc[start_date, :]

In [41]:
# Plot index
px.line(
    data_frame = index_df,
    title = 'Historical Index Based on Daily Adj. Close Prices',
    height = 600,
    labels = {'variable': 'Ticker', 'value': 'Unit'}
)